In [5]:
import pandas as pd
import numpy as np
import networkx as nx
from netgraph import Graph

df = pd.read_csv('queryResult-2008.csv', index_col=0)

G1 = nx.Graph()
G2 = nx.Graph()

nx.DiGraph()

"""
for index, row in df.iterrows():
    print(index, row['A'], row['B'])
"""

for index, row in df.iterrows():
    affil_list = str(row['affilname']).lower().split(';')
    for affil_from in affil_list:
        for affil_to in affil_list:
            if affil_from == affil_to:
                continue
            if G1.has_edge(affil_from, affil_to):
                G1[affil_from, affil_to]['weight'] += 0.1
            else:
                G1.add_edge(affil_from, affil_to, weigt=0.1)


for index, row in df.iterrows():
    country_list = str(row['affiliation_country']).lower().split(';')
    for country_from in country_list:
        for country_to in country_list:
            if country_from == country_to:
                continue
            if G2.has_edge(country_from, country_to):
                G2[country_from][country_to]['weight'] += 0.1
            else:
                G2.add_edge(country_from, country_to, weigt=0.1)

nx.write_gexf(G1, 'graph_institution.gexf')
nx.write_gexf(G2, 'graph_country.gexf')

KeyError: 'weight'